# Session 19: Tools for Big Data

Today you will be doing a few exercises with parallel processing using Python's `multiprocessing` package, and a couple others that bends your mind into thinking inside the MapReduce paradigm.

## Exercises

### Part 1: Multiprocessing

>**Ex. 19.1.1**: Explain the code from my lecture. Put a comment above each line of code and explain what happens.

In [1]:
from multiprocessing import Pool

def worker(x):
    return x * (x + 3)

p = Pool(4)  # <-- what does number mean?

inputs = range(10)
outputs = []

for result in p.imap(worker, inputs):  # <-- why `p.imap` and not just `p.map`?
    outputs.append(result)
    
p.close()

print(outputs)

[0, 4, 10, 18, 28, 40, 54, 70, 88, 108]


>**Ex. 19.1.2**: Do the same as above in serial, with a single line of code.<br>
> *Hint: Use a list comprehension*

>**Ex. 19.1.3**: Take the following bit of code and parallelize it. Report the speedup (if any)

In [26]:
from sklearn import datasets
import numpy as np
from datetime import datetime as dt

iris = datasets.load_iris().data

start = dt.now()

pairwise_distances = []
for i in range(iris.shape[0]):
    for j in range(i+1, iris.shape[0]):
        pairwise_distances.append(np.sqrt(sum((iris[i] - iris[j])**2)))

print("Time:", (dt.now() - start).total_seconds())

Time: 0.055428


### Part 2: Running your first MapReduce job

Just to get started right, lets make sure you're all setup with `mrjob` (you should be able to install it with `conda`, otherwise try `pip`), so you can write MapReduce jobs in Python. I'm gonna have you run a very simple MapReduce job.

>**Ex. 19.2.1**: Follow the example on the last slide of today's lecture and run your very first MapReduce job! You should get the same output as what is shown on the slide. As your answer, insert all of the terminal output in a markdown cell below, indented by one tab (mark all of the text, hit tab so it turns orange, and hit enter).

>*Hints:*

>1. Create a `.py` file with the content:
            
>        from mrjob.job import MRJob
> 
>        class MRWordCounter(MRJob):  
>
>            def mapper(self, _, line):
>                for word in line.split():
>                    yield word, 1
>
>            def reducer(self, key, values):
>                yield key, sum(values)
>
>        if __name__ == '__main__':
>            MRWordCounter.run()

>2. Create a `.txt` file with the content:
>        i am a twitter bot
>        i am also a twitter bot
>        wow such bot tweet also
>        very bot like tweet also

>3. Run the `.py` script with the `.txt` file as its argument in your terminal/console.

### Crash course: *Generators*

*Skip this section if you know everything about generators*.

Here follows a small, non-exhaustive, technical crash course in Python generators. By now you may have noticed that the "functions" inside the `mrjob` class have `yield` statement rather than the good old `return` statements that you are familiar with. What does this mean?

Python functions with `yield` statements are in fact not functions at all, they are *generators*. In brief, generators are a special type of function that *yield* output everytime the interpreter reaches a `yield` statement, and then, remembering where it left off, continues from that point the next time the generator instance is called to give output.

#### How generators work
Let's define a generator called `my_generator()`:

In [2]:
def my_generator():
    yield "first call"
    yield "second call"
    yield "final call"

Let's make an instance of this generator. We call such an instance an *iterator*.

In [15]:
my_iterator = my_generator()

Using the `.next()` method, we are then going to ask our new iterator object to give us some output.

In [16]:
print my_iterator.next()

first call


So this prints the output of the first `yield` statement that we put inside the generator. Let's do it again.

In [17]:
print my_iterator.next()

second call


AHA! Evidently, when calling it again, the interpreter continued from right after the first `yield`, and went straight to the next. So if we run `.next()` on it again it's surely going to give us the last string.

In [18]:
print my_iterator.next()

final call


And that's it! Try to call `.next()` on it again and think about why it fails. You can also play around with other generators, for example try putting a `yield` statement inside a `for` loop and think about the output you get.
>What you should realize, is that an iterator (recall: an instance of a generator) is like a tape that plays a series of `yield`s before it is over.

#### Generators in `mrjob`

Both the `mapper` and `reducer` are generators. Let's analyze the `mapper` generator from Ex. 19.2.1.

    def mapper(self, _, line):
        for word in line.split():
            yield word, 1
A *line* of text runs though it, gets split into words and `yield`ed into tuples that contain each word and a one. Let's say the line is "i am a string a short string" and code up an example:

In [23]:
def mapper(line):
    for word in line.split():
        yield word, 1

for output in mapper("i am a string a short string"):
    print output

('i', 1)
('am', 1)
('a', 1)
('string', 1)
('a', 1)
('short', 1)
('string', 1)


I removed the `self` and `_` arguments from the generator, just so that it works outside of a `class`, but what happens inside hasn't changed. Really, REALLY try to make sense of this code, because it is key to understanding how MapReduce works in Python!
>You should understand that when you put some data into the `mapper`, out of it comes a *sequence* of key-value pairs.

This entirely congruent with the figure I showed you in the lecture:

<img src="http://ulfaslak.com/computational_analysis_of_big_data/slide_figures/mapreduce_example.png" width="400"></img>

Although the arguments to the `reducer` look slightly different, it works exactly the same. It operates on one key-values pair (note that *values* is plural now) at a time, where the key is whatever you defined it to be in the `mapper` and the values are itself an iterator over all the different values that were paired with the key in the implicitly handled *group-by-key* step. The `reducer` can even have multiple `yield` statements, one for each operation that you want to make on the values. It's also possible to put the output of the `reducer` into a new `mapper` and put the output of that `mapper` into an even newer `reducer`, such as to make multi-step MapReduce operations – the possibilities are endless!

>Before moving on, I encourage you to play around with the script from Ex. 19.2.1, by insert some print statements here and there, deliberately breaking it, changing the output(s) of the `mapper` and the `reducer`, and so on. 

*Nerdy sidenote*: You could argue that the developers of `mrjob` might aswell have written their API so that the `mapper` and `reducer` were standard functions that returned a list of key-value pairs, rather than generators that produce them in sequence. The reason they chose generators is that they are much faster and have a smaller memory footprint.



### Part 3: Some serious MapReduce

Alright... Time for action. In this part of today's exercises you are going to apply your freshly acquired understanding of MapReduce to do some cool highly scalable computations. All the code you write here works as well on your computer as it does on a massively parallelized Hadoop data storage system, so while you may not experience serious speedups locally, you would if the code was deployed on e.g. some of Google's data storage clusters.

>**Ex. 19.3.1**: Modify the script from Ex. 19.2.1 so that it instead of word counts outputs the number of characters, the number of words and the numbers of lines in the input file. Post as your answer in two seperate cells, (1) the code in the script in a code cell, and (2) the terminal output in a markdown cell with the text indented by one tab.

>**Ex. 19.3.2**: Implement the "Common friends" example from today's lecture. Here is your data, you should put that into a file:

>        A B C D
>        B A C D E
>        C A B D E
>        D A B C E
>        E B C D
    
>I removed all but the letters, but you already know that the first letter in each line is a person and the following are her friends. Report your answer in the same fashion as above.

>**Ex. 19.3.3**: Lets go a bit deeper. In this exercise you will implement a MapReduce-MapReduce operation, which computes the same thing as we computed above, but takes as input friend-network data in a slightly more common format: 

>        A B
>        A C
>        A D
>        B C
>        B D
>        B E
>        C D
>        C E
>        D E

>Each line is a "friend-link". The links are undirected and each only occurs once.

>Your job now, is to produce the same output as you did in Ex. 19.3.2, using this input data. To get started faster, use the template below, which shows how to chain together multiple MapReduce steps. Fill out the template and show the output that you get from the terminal when you run it. Clarify whether it corresponds with the output from Ex. 19.3.2.

>*Hint: Try to write the first MapReduce step such that it outputs key-value pairs that correspond to the input data format from Ex. 19.3.2. Then you can reuse your solution to that exercise in the second MapReduce step.*

In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep

class NumberOfTriangles(MRJob):

    def mapper1(self, _, line):
        #

    def reducer1(self, key, values):
        #

    def mapper2(self, key, values):
        #

    def reducer2(self, key, values):
        #

    def steps(self):
        return [
            MRStep(
                mapper=self.mapper1,
                reducer=self.reducer1
            ),
            MRStep(
                mapper=self.mapper2,
                reducer=self.reducer2
            )
        ]

if __name__ == '__main__':
    NumberOfTriangles.run()

>**Ex. 19.3.4**: We can go even further! Let's add a third MapReduce step to count the number of triangles in a network. Again use this input data:

>        A B
>        A C
>        A D
>        B C
>        B D
>        B E
>        C D
>        C E
>        D E

>to validate that your implementation works. It should produce 7 triangles.

>1. Now compute the number of triangles in [this file](http://snap.stanford.edu/data/facebook_combined.txt.gz) which contains 88234 links in an anonymized facebook network. Don't print the whole output, just report the number you get.
>2. Do the same instead using all 2766607 road segments in California as your input. Go to [this site](https://www.cise.ufl.edu/research/sparse/matrices/SNAP/roadNet-CA.html) and download the data in Matrix Market format (`.mtx`). Unzip the file and remove the first 50 lines from it, since that is just markup that we don't need. The file is pretty big so you can expect it to take some time (~4 minutes on my computer). Report the number you get.

>*Hint: Counting triangles is equivalent to counting "common friends". One way to do that is to just count the collective number of common friends that exist in a network. Depending on your implementation you might want to correct your result by a factor 3, since it is likely that you end up counting each triangle three times (one for each point in it).*

>*Nerdy sidenote: Why would anyone want to count triangles??? Well, in network science there is a lot of statistical measures that include the count of triangles in a network. For example, the [clustering coefficient](https://en.wikipedia.org/wiki/Clustering_coefficient), which reveals the proportion of small closed loops in a network, is computed as the number of realized triangles divided by the number of possible triangles.*